In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
import tensorflow as tf
print(tf.__version__)
import keras
print(keras.__version__)

/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sunxin/minico

1.13.2
2.2.4


Using TensorFlow backend.


In [2]:
import numpy as np
from keras import models
from keras import layers

In [3]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = '../datasets/fra-eng/fra.txt'

In [4]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [5]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [6]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [7]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [8]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [9]:
# Define an input sequence and process it.
encoder_inputs = layers.Input(shape=(None, num_encoder_tokens))
# Encoder
x_encoder = layers.Conv1D(256, kernel_size=3, activation='relu',
                          padding='causal')(encoder_inputs)
x_encoder = layers.Conv1D(256, kernel_size=3, activation='relu',
                          padding='causal', dilation_rate=2)(x_encoder)
x_encoder = layers.Conv1D(256, kernel_size=3, activation='relu',
                          padding='causal', dilation_rate=4)(x_encoder)

decoder_inputs = layers.Input(shape=(None, num_decoder_tokens))
# Decoder
x_decoder = layers.Conv1D(256, kernel_size=3, activation='relu',
                          padding='causal')(decoder_inputs)
x_decoder = layers.Conv1D(256, kernel_size=3, activation='relu',
                          padding='causal', dilation_rate=2)(x_decoder)
x_decoder = layers.Conv1D(256, kernel_size=3, activation='relu',
                          padding='causal', dilation_rate=4)(x_decoder)

# Attention
attention = layers.Dot(axes=[2, 2])([x_decoder, x_encoder])
attention = layers.Activation('softmax')(attention)

context = layers.Dot(axes=[2, 1])([attention, x_encoder])
decoder_combined_context = layers.Concatenate(axis=-1)([context, x_decoder])

decoder_outputs = layers.Conv1D(64, kernel_size=3, activation='relu',
                                padding='causal')(decoder_combined_context)
decoder_outputs = layers.Conv1D(64, kernel_size=3, activation='relu',
                                padding='causal')(decoder_outputs)
# Output
decoder_dense = layers.Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, 93)     0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None, 70)     0                                            
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, None, 256)    71680       input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, None, 256)    54016       input_1[0][0]                    
_____________________________________

In [10]:
# Run training
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('cnn_s2s.h5')

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 5s 569us/step - loss: 0.9045 - val_loss: 0.7947
Epoch 2/100
8000/8000 [==============================] - 2s 221us/step - loss: 0.6105 - val_loss: 0.6586
Epoch 3/100
8000/8000 [==============================] - 2s 225us/step - loss: 0.5248 - val_loss: 0.5967
Epoch 4/100
8000/8000 [==============================] - 2s 219us/step - loss: 0.4764 - val_loss: 0.5611
Epoch 5/100
8000/8000 [==============================] - 2s 224us/step - loss: 0.4410 - val_loss: 0.5428
Epoch 6/100
8000/8000 [==============================] - 2s 224us/step - loss: 0.4158 - val_loss: 0.5177
Epoch 7/100
8000/8000 [==============================] - 2s 222us/step - loss: 0.3930 - val_loss: 0.5110
Epoch 8/100
8000/8000 [==============================] - 2s 226us/step - loss: 0.3763 - val

8000/8000 [==============================] - 2s 222us/step - loss: 0.0538 - val_loss: 0.8872
Epoch 73/100
8000/8000 [==============================] - 2s 225us/step - loss: 0.0536 - val_loss: 0.8829
Epoch 74/100
8000/8000 [==============================] - 2s 221us/step - loss: 0.0537 - val_loss: 0.8832
Epoch 75/100
8000/8000 [==============================] - 2s 223us/step - loss: 0.0508 - val_loss: 0.8891
Epoch 76/100
8000/8000 [==============================] - 2s 225us/step - loss: 0.0529 - val_loss: 0.8885
Epoch 77/100
8000/8000 [==============================] - 2s 224us/step - loss: 0.0515 - val_loss: 0.8902
Epoch 78/100
8000/8000 [==============================] - 2s 223us/step - loss: 0.0503 - val_loss: 0.8804
Epoch 79/100
8000/8000 [==============================] - 2s 226us/step - loss: 0.0484 - val_loss: 0.8933
Epoch 80/100
8000/8000 [==============================] - 2s 221us/step - loss: 0.0477 - val_loss: 0.8965
Epoch 81/100
8000/8000 [==============================] - 2

In [11]:
# Next: inference mode (sampling).

# Define sampling models
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [12]:
nb_examples = 100
in_encoder = encoder_input_data[:nb_examples]
in_decoder = np.zeros(
    (nb_examples, max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
in_decoder[:, 0, target_token_index["\t"]] = 1

for i in range(max_decoder_seq_length - 1):
    predict = model.predict([in_encoder, in_decoder])
    predict = predict.argmax(axis=-1)
    predict_ = predict[:, i].ravel().tolist()
    for j, x in enumerate(predict_):
        in_decoder[j, i + 1, x] = 1

for seq_index in range(nb_examples):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    output_seq = predict[seq_index, :].ravel().tolist()
    decoded = []
    for x in output_seq:
        if reverse_target_char_index[x] == "\n":
            break
        else:
            decoded.append(reverse_target_char_index[x])
    decoded_sentence = "".join(decoded)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    print('Target sentence:', target_texts[seq_index][1:-1])

-
Input sentence: Go.
Decoded sentence: Va !
Target sentence: Va !
-
Input sentence: Hi.
Decoded sentence: Salut.
Target sentence: Salut !
-
Input sentence: Hi.
Decoded sentence: Salut.
Target sentence: Salut.
-
Input sentence: Run!
Decoded sentence: Courez !
Target sentence: Cours !
-
Input sentence: Run!
Decoded sentence: Courez !
Target sentence: Courez !
-
Input sentence: Who?
Decoded sentence: Qui ?
Target sentence: Qui ?
-
Input sentence: Wow!
Decoded sentence: Ça alors !
Target sentence: Ça alors !
-
Input sentence: Fire!
Decoded sentence: Au feu !
Target sentence: Au feu !
-
Input sentence: Help!
Decoded sentence: À l'aide !
Target sentence: À l'aide !
-
Input sentence: Jump.
Decoded sentence: Saute.
Target sentence: Saute.
-
Input sentence: Stop!
Decoded sentence: Arrête-toi !
Target sentence: Ça suffit !
-
Input sentence: Stop!
Decoded sentence: Arrête-toi !
Target sentence: Stop !
-
Input sentence: Stop!
Decoded sentence: Arrête-toi !
Target sentence: Arrête-toi !
-
Input se